In [ ]:
# 환경설정
from gekko import GEKKO
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import matplotlib
import matplotlib.pyplot as plt
df = pd.read_excel("C:/Users/GACHON/OneDrive/바탕 화면/입력데이터_.xlsx")
m=GEKKO()
df

ModuleNotFoundError: ignored

In [ ]:
K = range(24)
dk = 1
P_L = np.array(df["Load[kWh]"],float) #부하
P_PV = np.array(df["PV_Generation[kWh]"],float) 
SMP = np.array(df["SMP[원/kWh]"],float)
TOU = np.array(df["TOU[원/kWh]"],float)

p_grid = [m.Var() for k in K] #Grid에서 가져온 전기
p_sold = [m.Var() for k in K] #Grid로 판매한 전기

p_c_e1 = [m.Var() for k in K] #ESS1 충전량
p_d_e1 = [m.Var() for k in K] #ESS1 방전량
soe_e1 = [m.Var() for k in K] #ESS1의 SOE상태
u_e1 = [m.Var(integer=True) for k in K]
u_g = [m.Var(integer=True) for k in K]

CE_ESS1 = 0.95 #ESS1 충전효율
DE_ESS1 = 0.95 #ESS1 방전효율

SOE_ESS1_init = 500 #ESS1 초기값
N1 = 15000 #계통제약
N2 = 15000

In [ ]:
#비음조건
m.Equation([p_c_e1[k] >= 0 for k in K])
m.Equation([p_d_e1[k] >= 0 for k in K])
m.Equation([soe_e1[k] >= 0 for k in K])

1. 계통 모델링

In [ ]:
m.Equation([P_L[k] + p_c_e1[k] + p_sold[k] == P_PV[k] + p_grid[k] + p_d_e1[k]*DE_ESS1  for k in K ])
m.Equation([p_grid[k]<= N1 * u_g[k] for k in K]) #그리드 이진변수
m.Equation([p_sold[k]<= N2 * (1-u_g[k]) for k in K])
m.Equation([ 0 <= p_grid[k] for k in K])
m.Equation([ 0 <= p_sold[k] for k in K])

2. ESS 모델링

In [ ]:
#ESS1의 SOE 조건
m.Equation([soe_e1[0] == SOE_ESS1_init + CE_ESS1*p_c_e1[0]- p_d_e1[0] ])
m.Equation([soe_e1[k] == soe_e1[k-1] + CE_ESS1*p_c_e1[k] - p_d_e1[k] for k in range(1,23)])
#SOE 제약조건
m.Equation([soe_e1[k] <= 950 for k in K])
m.Equation([100 <= soe_e1[k] for k in K])
# m.Equation([soe_e1[23] <= 700 for k in K])
# m.Equation([100 <= soe_e1[23] for k in K])
# 충방전 제약조건
# m.Equation([10 <= p_c_e1[k] for k in K ])
# m.Equation([p_c_e1[k] <= 500 for k in K ])
# m.Equation([ 10 <= p_d_e1[k] for k in K ])
# m.Equation([p_d_e1[k] <= 500 for k in K ])
m.Equation([u_e1[k]*10 <= p_c_e1[k] for k in K ])
m.Equation([p_c_e1[k] <= 500*u_e1[k] for k in K ])
m.Equation([( 1-u_e1[k]) * 10<=p_d_e1[k] for k in K ])
m.Equation([p_d_e1[k] <= 500* (1-u_e1[k]) for k in K ])

3. 수익

In [ ]:
revenue = sum(p_sold[k]*SMP[k] for k in K)

In [ ]:
cost = sum (p_grid[k]*TOU[k] +(CE_ESS1*p_c_e1[k] + p_d_e1[k]) * 2 for k in K)

In [ ]:
m.Obj(cost)
m.options.SOLVER=1
m.solve()

apm 203.249.127.37_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          480
   Intermediates:            0
   Connections  :            0
   Equations    :          362
   Residuals    :          362
 
 Number of state variables:            480
 Number of total equations: -          359
 Number of slack variables: -          312
 ---------------------------------------
 Degrees of freedom       :           -191
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.03 NLPi:    5 Dpth:    0 Lvs:    3 Obj:  1.75E+04 Gap:       NaN
Iter:     2 I:

In [ ]:
result=pd.DataFrame({'p_c_e1': np.round(np.array(p_c_e1). flatten(),2),
                     'p_d_e1': np.round(np.array(p_d_e1). flatten(),2),
                     'p_grid': np.round(np.array(p_grid). flatten(),2),
                     'p_sold': np.round(np.array(p_sold). flatten(),2),
                     'soe_e1': np.round(np.array(soe_e1). flatten(),2),
                     'u_e1': np.round(np.array(u_e1). flatten(),2),
                     'u_g': np.round(np.array(u_g). flatten(),2)})

In [ ]:
result

,p_c_e1,p_d_e1,p_grid,p_sold,soe_e1,u_e1,u_g
0,64.80,0.00,110.80,0.00,561.56,1.0,1.0
1,64.80,0.00,111.80,0.00,623.13,1.0,1.0
2,64.81,0.00,109.81,0.00,684.69,1.0,1.0
3,64.83,0.00,109.83,0.00,746.28,1.0,1.0
4,64.81,0.00,110.81,0.00,807.85,1.0,1.0
5,64.81,0.00,111.81,0.00,869.42,1.0,1.0
6,64.82,0.00,108.82,0.00,931.00,1.0,1.0
7,10.00,0.00,46.00,0.00,940.50,1.0,1.0
8,10.00,0.00,36.00,0.00,950.00,1.0,1.0
9,0.00,10.00,0.00,7.50,940.00,0.0,0.0


In [ ]:
result.to_excel("result_1.1.xlsx")